Import libraries

In [1]:
import os
import tensorflow as tf
import argparse
import miscnn
import random
import SimpleITK as sitk
from miscnn.data_loading.interfaces.dicom_io import DICOM_interface
from miscnn.neural_network.data_generator import DataGenerator
from miscnn.data_loading.data_io import Data_IO 
from miscnn.utils.visualizer import normalize_volume , visualize_samples , detect_dimensionality
from miscnn.processing.data_augmentation import Data_Augmentation
from miscnn.processing.subfunctions import Normalization, Clipping, Resampling
from miscnn import Preprocessor, Neural_Network
from miscnn.neural_network.metrics import dice_soft, tversky_crossentropy,categorical_focal_loss, dice_crossentropy, asym_unified_focal_loss
from miscnn.neural_network.architecture.unet.dense import Architecture

from miscnn.evaluation.cross_validation import cross_validation, write_fold2disk, split_folds, run_fold, load_disk2fold
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, \
                                       EarlyStopping, CSVLogger, ModelCheckpoint
from miscnn.evaluation import split_validation
from IPython.display import Image
import csv
import matplotlib.pyplot as plt
import numpy as np

Tensorflow and set GPU

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Argparser

In [3]:
#Create a fold directory for k-fold for Cross-Validation

# Number of folds
k_folds = 5

# Base directory for evaluation
base_dir = "evaluation"

# Create the base directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Create subdirectories for each fold
for fold in range(1, k_folds + 1):
    fold_subdir = os.path.join(base_dir, "fold_" + str(fold))
    if not os.path.exists(fold_subdir):
        os.makedirs(fold_subdir)
        print(f"Created directory: {fold_subdir}")
    else:
        print(f"Directory already exists: {fold_subdir}")


DICOM interface

In [4]:
# Create Interface
interface = DICOM_interface(annotation_tag="rtstruct")

# Initialize 
data_path = "/restricted/projectnb/ortho-ar/MinKim/data_teeth"
samples = interface.initialize(data_path)

# Obtain ROI names
structures = interface.get_ROI_names(samples[0])
print('Found structures in sample : {}'.format(structures))

Found structures in sample : [["1", 'Maxillary'], ["2", 'Mandible']]


Assign Class to Structure

In [5]:
# Class 0 is always background class 
structure_dict = {"Maxillary": 1, "Mandible": 2}

print(structure_dict)

{'Maxillary': 1, 'Mandible': 2}


Create New Instance of the DICOM interface with Structure Dictionary and Number of Classes

In [6]:
interface = DICOM_interface(structure_dict = structure_dict, classes=3, annotation_tag="rtstruct")

Connect the DICOM interface to the DATA_IO class

In [7]:
data_io = Data_IO(interface, data_path, "/restricted/projectnb/ortho-ar/MinKim/predictions/predicted_teeth")

check the interface by getting a list of all samples

In [ ]:
sample_list = data_io.get_indiceslist()
sample_list.sort()
sample_number = len(sample_list)
print("All samples: " + str(sample_list))
print("Number of Samples: " + str(sample_number))

Distribute Training & Validation

In [9]:
write_fold2disk(file_path= "/restricted/projectnb/ortho-ar/MinKim/evaluation/fold_5/sample_list.json", training = sample_list[0:247], validation = sample_list[247:])

Checking the functionality of the data pipeline 

In [ ]:
sample = data_io.sample_loader(samples[1])
images, segmentations = sample.img_data, sample.seg_data

print(images.shape, segmentations.shape)

Set Up Data Augmentation

In [11]:
# Create and configure the Data Augmentation class
data_aug = Data_Augmentation(cycles=2, scaling=True, rotations=True, elastic_deform=True, mirror=True,
                             brightness=True, contrast=True, gamma=True, gaussian_noise=True)

Find out good voxel spacing

In [ ]:
sf_resample = Resampling((1.8, 2, 2))
pp_test = Preprocessor(data_io, data_aug=None, batch_size=1, subfunctions=[sf_resample], 
                       prepare_subfunctions=True, prepare_batches=False, 
                       analysis="fullimage")

data_gen = DataGenerator(sample_list, pp_test, training=False,
                         shuffle=False, iterations=None)
x = []
y = []
z = []
for batch in data_gen:
    x.append(batch.shape[1])
    y.append(batch.shape[2])
    z.append(batch.shape[3])

# Clean up batch directory, afterwards
data_io.batch_cleanup()

print(np.median(x), np.median(y), np.median(z))

Subfunctions for processing the data

In [12]:
# Create a pixel value normalization Subfunction through Z-Score 
sf_normalize = Normalization(mode="grayscale")
# Create a resampling Subfunction to voxel spacing 1 x 1 x 1
sf_resample = Resampling((1, 1, 1))
# Create a clipping Subfunction to the teeth window of CTs 
sf_clipping = Clipping(min=-300, max=3000)
# Create a pixel value normalization subfunction for z-score scaling
sf_zscore = Normalization(mode="z-score")

# Assemble Subfunction classes into a list
# Be aware that the Subfunctions will be exectued according to the list order!
subfunctions = [sf_clipping, sf_normalize, sf_resample, sf_zscore]

Create preprocessor and Decide patchwise or full image training for CNN

In [13]:
# Create and configure the Preprocessor class
pp = Preprocessor(data_io, data_aug=data_aug, batch_size=3, subfunctions=subfunctions, prepare_subfunctions=True, 
                  prepare_batches=False, analysis="patchwise-crop", patch_shape=(80, 160, 160))

pp.patchwise_overlap = (40, 80, 80)

Building the Neural Network (DENSE)

In [ ]:
# Initialize the Architecture
unet_standard = Architecture(depth=4, activation="softmax",
                             batch_normalization=True)

# Create the Neural Network model with the legacy optimizer
model = Neural_Network(preprocessor=pp, loss=tversky_crossentropy, metrics=[dice_soft], batch_queue_size=3, workers=3, learning_rate=0.00001)



In [19]:
model.reset()
model.reset_weights()

Load 3D UNET Network Architecture from MIScnn

In [20]:
from miscnn.neural_network.architecture.unet.standard import Architecture

# Initialize the Architecture
unet_standard = Architecture(depth=4, activation="softmax",
                             batch_normalization=True)

model = Neural_Network(preprocessor=pp, architecture=unet_standard,
                       loss=tversky_crossentropy,
                       metrics=[dice_soft],
                       batch_queue_size=3, workers=3, learning_rate=0.0001)


Callbacks

In [21]:
# Define Callbacks
cb_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=20,
                          verbose=1, mode='min', min_delta=0.0001, cooldown=1,
                          min_lr=0.000001)
cb_es = EarlyStopping(monitor="loss", patience=50)
cb_tb = TensorBoard(log_dir=os.path.join(fold_subdir, "tensorboard"),
                    histogram_freq=0, write_graph=True, write_images=True)
cb_cl = CSVLogger(os.path.join(fold_subdir, "logs.csv"), separator=',',
                  append=True)
cb_mc = ModelCheckpoint(os.path.join(fold_subdir, "model.best.hdf5"),
                        monitor="loss", verbose=1,
                        save_best_only=True, mode="min")

Run pipeline for CV fold

In [ ]:
#-----------------------------------------------------#
#          Run Pipeline for provided CV Fold          #
#-----------------------------------------------------#
run_fold(fold, model, epochs=200, iterations=200, evaluation_path="evaluation",
         draw_figures=True, callbacks=[cb_lr, cb_es, cb_tb, cb_cl, cb_mc],
         save_models=False)
# Run pipeline for cross-validation fold

# Dump latest model to disk
model.dump(os.path.join(fold_subdir, "model.latest.hdf5"))

Inference for provided CV fold

In [ ]:
#-----------------------------------------------------#
#           Inference for provided CV Fold            #
#-----------------------------------------------------#


# Load best model weights during fitting
model.load(os.path.join(fold_subdir, "model.best.hdf5"))

# Obtain training and validation data set
training, validation = load_disk2fold(os.path.join(fold_subdir,
                                                   "sample_list.json"))

# Compute predictions
model.predict(validation, return_output=False)


Check Training/ Validation Dice Soft 

In [ ]:

Image(filename = "evaluation/fold_5/validation.dice_soft.png")

Check Training / Validation Loss

In [ ]:
Image(filename = "evaluation/fold_5/validation.loss.png")

Reset DICOM interface with directory to data to be predicted

In [47]:
data_path = "/restricted/projectnb/ortho-ar/MinKim/predictions/teeth_predict"
data_io = Data_IO(interface, data_path, "/restricted/projectnb/ortho-ar/MinKim/predictions/predicted_teeth")

In [ ]:
sample_list = data_io.get_indiceslist()
sample_list.sort()
sample_number = len(sample_list)
print("All samples: " + str(sample_list))
print("Number of Samples: " + str(sample_number))

Load the trained model and create prediction with 3D UNet

In [51]:

# Create the Neural Network model with the legacy optimizer
model = Neural_Network(preprocessor=pp, loss=tversky_crossentropy, metrics=[dice_soft], batch_queue_size=3, workers=3, learning_rate=0.000001)

model.load("/restricted/projectnb/ortho-ar/MinKim/Model/model_5.hdf5")
output_path = "/restricted/projectnb/ortho-ar/MinKim/predictions/predicted_teeth"
model.predict(sample_list)
